### Manipulating Sequences — Advanced

This advanced notebook extends sequence manipulation beyond basic indexing/slicing. It covers:

- In-place vs. new-object operations (identity & aliasing)
- Concatenation vs. `extend` (and performance/semantics)
- Slice assignment idioms (insert/replace/delete/stride)
- `del` on slices (bulk deletions)
- `append`, `extend`, `insert`, `pop`, `remove`, `clear`
- Multiplication pitfalls with nested lists
- Sorting: `list.sort()` vs `sorted()`; stability, `key=`, `reverse=`
- Copying lists: `[:]`, `.copy()`, `list()`, and `copy.deepcopy`
- `bytearray` (mutable string-like sequence) & `memoryview`
- Queues/stacks: `list` vs `collections.deque`
- Safe insert/search with `bisect`
- Quick micro-benchmarks with `timeit` (no plots)

All examples are pure Python and safe to run in a fresh kernel.

## 1) In-place vs new-object operations
- Some operations mutate the **same** list (in-place) and keep its identity.
- Others allocate a **new** object.

Use `is` to check identity.

In [2]:
l = [1, 2, 3]
same_id_before = id(l)
l += [4, 5]            # in-place extend for lists
same_id_after_plus_eq = (id(l) == same_id_before)

l2 = [1, 2, 3]
same_id_before2 = id(l2)
l2 = l2 + [4, 5]       # creates a new list
        new_object_created = (id(l2) != same_id_before2)
same_id_after_plus_eq, new_object_created, l, l2

IndentationError: unexpected indent (2428078543.py, line 9)

Takeaway: `+=` on lists is generally in-place (like `extend`), while `+` returns a new object (and may copy more data).

## 2) Concatenation vs `extend`
- `l + r` → new list; `l += r` or `l.extend(r)` → mutate `l`.
- `extend` iterates over the right-hand iterable and appends each element.
- Use `append` for a **single** object; `extend` for many; avoid `append(iterable)` unless you actually want a nested object.

In [3]:
l = [1, 2]
l.append([3, 4])  # a single object (a list) appended
nested = l.copy()

l = [1, 2]
l.extend([3, 4])  # 3 and 4 are appended
flat = l.copy()

nested, flat

([1, 2, [3, 4]], [1, 2, 3, 4])

## 3) Slice assignment idioms (insert/replace/delete/stride)
Slice assignment lets you edit multiple elements at once. With a **step** specified, RHS length must match the number of targeted positions.

In [4]:
# Replace a contiguous region (lengths can differ)
l = [0,1,2,3,4,5]
l[2:5] = [20,30]        # shrink
rep1 = l.copy()

# Insert via empty slice
l[2:2] = ['A','B']      # insertion at index 2
rep2 = l.copy()

# Delete via empty RHS
l[1:4] = []
rep3 = l.copy()

# Stride replacement — lengths must match
        l = [0,1,2,3,4,5,6]
l[::2] = [10,11,12,13]  # replace indices 0,2,4,6
rep4 = l.copy()
rep1, rep2, rep3, rep4

IndentationError: unexpected indent (3792199994.py, line 15)

**`del` with slices** removes many items at once (lists only).

In [5]:
l = list(range(10))
del l[::3]   # delete indices 0,3,6,9
l  # [1,2,4,5,7,8]

[1, 2, 4, 5, 7, 8]

## 4) Element-level methods: `append`, `insert`, `extend`, `pop`, `remove`, `clear`
- `append(x)`: push one object `x`
- `insert(i, x)`: insert at index `i` (O(n))
- `extend(iterable)`: append all items
- `pop([i])`: remove and return item at `i` (default last)
- `remove(x)`: remove first occurrence of `x` (ValueError if not found)
- `clear()`: remove all items (same as `del l[:]`)

Note: inserting near the front of a list is slower than appending at the end (due to shifting elements).

In [6]:
l = [1, 2, 3]
l.insert(1, 'X')   # [1, 'X', 2, 3]
mid = l.copy()
last = l.pop()     # removes 3
after_pop = l.copy()
l.remove('X')      # remove first 'X'
after_remove = l.copy()
l.clear()          # []
mid, last, after_pop, after_remove, l

([1, 'X', 2, 3], 3, [1, 'X', 2], [1, 2], [])

## 5) Multiplication pitfalls with nested lists
Multiplying a list repeats **references** to the same inner objects (shallow replication), which can cause surprising shared-mutation bugs with nested lists.

In [7]:
row = [0, 0, 0]
m = [row] * 3          # three references to the same row
m[1][1] = 99
bad = m                 # every row appears changed in the same column

# Correct way: list comprehension to create distinct rows
m2 = [[0,0,0] for _ in range(3)]
m2[1][1] = 99
good = m2
bad, good

([[0, 99, 0], [0, 99, 0], [0, 99, 0]], [[0, 0, 0], [0, 99, 0], [0, 0, 0]])

## 6) Sorting sequences
- `sorted(iterable, *, key=None, reverse=False)` returns a **new** list.
- `list.sort(*, key=None, reverse=False)` sorts **in place** and returns `None`.
- Python's sort is **stable** (equal keys keep original order). Useful for multi-key sorts (sort by secondary key first, then primary).

In [8]:
data = [
    {"name": "Ana", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Bob", "age": 20},
    {"name": "Eve", "age": 25},
]

# Multi-key stable sort: secondary age asc, then primary name asc
by_age_then_name = sorted(sorted(data, key=lambda d: d["age"]), key=lambda d: d["name"]) 

# Or use a tuple key in one pass
tuple_key = sorted(data, key=lambda d: (d["name"], d["age"]))
by_age_then_name, tuple_key

([{'name': 'Ana', 'age': 30},
  {'name': 'Bob', 'age': 20},
  {'name': 'Bob', 'age': 25},
  {'name': 'Eve', 'age': 25}],
 [{'name': 'Ana', 'age': 30},
  {'name': 'Bob', 'age': 20},
  {'name': 'Bob', 'age': 25},
  {'name': 'Eve', 'age': 25}])

Reverse numeric sort and custom key example:

In [9]:
nums = [10, -3, 7, 2, -20]
descending = sorted(nums, reverse=True)
by_abs = sorted(nums, key=abs)
descending, by_abs

([10, 7, 2, -3, -20], [2, -3, 7, 10, -20])

## 7) Copying lists correctly
- Shallow copies:
  - `l_copy = l[:]`
  - `l_copy = list(l)`
  - `l_copy = l.copy()`
- Deep copies (nested structures):
  - `copy.deepcopy(l)`

Shallow copies duplicate the **outer** list but **share** inner objects. Use deep copy for independent nested copies (slower).

In [10]:
import copy
nested = [[1], [2]]
shallow = nested[:]          # shares inner lists
deep = copy.deepcopy(nested)
nested[0].append(99)
nested, shallow, deep  # shallow reflects change; deep does not.

([[1, 99], [2]], [[1, 99], [2]], [[1], [2]])

## 8) `bytearray` and `memoryview`
`bytearray` is a **mutable** sequence of integers in range 0..255 (bytes). `bytes` is immutable.
`memoryview` provides a zero-copy view of bytes-like objects (including slices) and can be used to mutate underlying buffers (when allowed).

In [11]:
b = bytearray(b"hello")
b[0] = ord('H')        # mutate first byte
mv = memoryview(b)
mv[1:5] = b"ELLO"      # mutate via view
bytes(b), bytes(mv)    # show as immutable bytes for display

(b'HELLO', b'HELLO')

Large binary manipulations benefit from `bytearray`/`memoryview` for fewer copies compared to string/bytes recreation.

## 9) Queues/Stacks: `list` vs `collections.deque`
- `list.append` and `list.pop()` (from the end) are amortized O(1) → good stack.
- Insert/remove at the **front** of a list is O(n).
- `collections.deque` provides O(1) append/pop **both ends** → better queue/Deque.

Use the right container for your access pattern.

In [12]:
from collections import deque
dq = deque([1,2,3])
dq.appendleft(0)
dq.append(4)
left = dq.popleft(); right = dq.pop()
list(dq), left, right  # remaining, left popped, right popped

([1, 2, 3], 0, 4)

## 10) Ordered insert/search with `bisect`
For sorted lists, `bisect` computes insertion points in O(log n) while insert remains O(n) (shift cost). Still, it's often cleaner/less error-prone than manual searches.

In [13]:
import bisect
a = [10, 20, 30, 40]
pos = bisect.bisect_left(a, 25)  # insertion point to keep order
a.insert(pos, 25)
a, pos  # [10, 20, 25, 30, 40], 2

([10, 20, 25, 30, 40], 2)

Lower/upper bounds:
- `bisect_left(a, x)` → first index where `x` could be inserted (before equal items)
- `bisect_right(a, x)` → after equal items (aka upper bound)

In [14]:
a = [10, 20, 20, 20, 30]
lo = bisect.bisect_left(a, 20)
hi = bisect.bisect_right(a, 20)
(lo, hi), a[lo:hi]  # range of equals

((1, 4), [20, 20, 20])

## 11) Micro-benchmarks (indicative only)
Use `timeit` for rough comparisons; avoid premature optimization. Here we compare `append` vs `insert(0, x)` into an ever-growing list (append wins for lists). Numbers will vary by machine/Python version.

*Note:* These are small, safe, terminal-only timings (no charts).

In [15]:
from timeit import timeit
append_time = timeit("l.append(1)", setup="l=[]", number=200_000)
insert_front_time = timeit("l.insert(0, 1)", setup="l=[]", number=50_000)
append_time, insert_front_time  # append is typically much faster for lists.

(0.015946200001053512, 1.1252600000007078)